<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2024-1/Insumos/BonoCupVarBondeD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 913, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 913 (delta 88), reused 99 (delta 64), pack-reused 768
Receiving objects: 100% (913/913), 62.96 MiB | 14.74 MiB/s, done.
Resolving deltas: 100% (593/593), done.
Updating files: 100% (417/417), done.


In [22]:
%%R
library(dplyr)
install.packages("data.table")
require(data.table)
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


(as ‘lib’ is unspecified)







	‘/tmp/RtmpbEodi0/downloaded_packages’



**Características**

Valor nominal: Generalmente, 100 pesos pero varía por emisora.

Plazo: de 1 a 360 días, según las necesidades de financiamiento de la empresa emisora.

Rendimiento: al igual que los bonos, este instrumento se compra a descuento respecto de su valor nominal, pero por lo general pagan una sobretasa referenciada a tasa fija o variable.

Garantía: este título, por ser un pagaré, no ofrece ninguna garantía, por lo que es importante evaluar bien al emisor.



In [23]:
%%R
btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"
btasadescst="RiesgosFinancieros/2020-1/Insumos/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-1/Insumos/tfondeo.txt"
fval=as.Date("20180629",format="%Y%m%d") #Fecha de valoración
plazos=cbind( 358,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon=cbind( 28,	28, 28, 28, 28) #plazos fijos de cada cupón
contratos=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal=100
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada


In [25]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1


data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20170629",format="%Y%m%d"), as.Date("20180629",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)
#tfhd

In [27]:

%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf]


In [42]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.


In [49]:
%%R
m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos


for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}



     [,1] [,2] [,3] [,4] [,5]
[1,]    0    0    0    0    0
           [,1]      [,2]       [,3]       [,4]       [,5]
[1,] 0.07804226 0.0769081 0.07764526 0.07830053 0.07661949
           [,1]       [,2]       [,3]       [,4]       [,5]
[1,] 0.07941412 0.07827887 0.07904407 0.07956142 0.07755682


In [57]:
%%R
#calcula cupones de bonos

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}

}




$$\Large V=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c_{p_i}}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$
	Donde: \\
$N$: Valor Nominal del bono\
$C$: Número de contratos\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c_{p_i}}$: Tasa cupón variable, se obtiene de la curva subyacente que le corresponda, casi siempre con la tasa forward entre $p_{i-1}$ y $p_i$.\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos según el plazo acumulado al pago del cupón $i$.


En general tenemos dos factores de riesgo subyacente (la curva de valor presente y la curva de cupones) pero como cada cupón tiene ``$n$'' flujos entonces tiene $n$ factores de riesgo para los valores presentes y $n$ factores de riesgo para los cupones, por lo que tienen $2n$ factores de riesgo específicos que provienen de dos factores de riesgo subyacentes.
Para el caso de $K$ bonos cupón variable el número de factores de riesgo sería $2\sum_{i=1}^{K}n_i$, donde $n_i$ es el número de cupones a pagar del bono $i$.

In [81]:
%%R

bondeD=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N)
{
  V0=matrix(0,1,nrow(as.matrix(N)))
  V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+(Xvp+Xst)*VTplazos/360)))*nominal
  for (j in (1:nrow(as.matrix(N))))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0=bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], Xst[1,], N)

V0

        [,1]     [,2]    [,3]     [,4]     [,5]
[1,] 2155549 -2840104 2817830 -4342999 933071.2


In [115]:
(930116.72280706/933071.2-1)*100
(-2834587.18559718/-2840104-1)*100

-0.19424691500100622

#Python traducción

In [24]:
import numpy as np
import pandas as pd
from datetime import datetime

btasadesc = "RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"
btasadescst = "RiesgosFinancieros/2020-1/Insumos/tasa_guber_st.txt"
btasafondeo = "RiesgosFinancieros/2020-1/Insumos/tfondeo.txt"

# Fecha de valoración
fval = datetime.strptime("20180629", "%Y%m%d")

# Parámetros de los bonos
plazos = np.array([358, 405, 550, 1200, 1800])
plazocupon = np.array([28, 28, 28, 28, 28])
contratos = np.array([22000, -29000, 29000, -46000, 10000])
nominal = 100
itpl = 0  # 0 para interpolación lineal, 1 para tasa alambrada


In [40]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Leer datos de tasa_guber.txt
data1 = pd.read_table(btasadesc, sep="\t", header=None)
n = data1.shape[0]
m_orig = data1.shape[1]
X_orig = data1.iloc[1:n, 1:m_orig]
X1_orig = data1.iloc[1:n, 0:m_orig]
X1_orig[0] = pd.to_datetime(X1_orig[0], format='%Y%m%d')
nodos = data1.iloc[0, 1:m_orig-1]
n -= 1

# Leer datos de tasa_guber_st.txt
data3 = pd.read_table(btasadescst, sep="\t", header=None)
n3 = data3.shape[0]
m3_orig = data3.shape[1]
X3_orig = data3.iloc[1:n3, 1:m3_orig]
X3a_orig = data3.iloc[1:n3, 0:m3_orig]
X3a_orig[0] = pd.to_datetime(X3a_orig[0], format='%Y%m%d')
nodos3 = data3.iloc[0, 1:m3_orig]
n3 -= 1

# # Leer datos de tfondeo.txt
data2 = pd.read_table(btasafondeo, sep="\t", header=None)
n2 = data2.shape[0]
X2_orig = data2.iloc[1:n2, 0:2]
X2a_orig = X2_orig.copy()
X2a_orig[0] = pd.to_datetime(X2a_orig[0], format='%Y%m%d')
X2a_orig[1] = X2a_orig[1].astype(float)

# # Sucesión de días para tasa fondeo
tfh = pd.date_range(start='2017-06-29', end='2018-06-29', freq='D')
tfhd = pd.DataFrame({'fecha': tfh})


In [41]:
# Asegúrate de que ambos DataFrames estén ordenados por la columna de fecha de manera descendente
tfhd = tfhd.sort_values(by='fecha', ascending=True)
X2a_orig = X2a_orig.sort_values(by=0, ascending=True)  # Asumiendo que la columna 'V1' es ahora la columna 0

# Renombrar columnas para preparar para merge_asof
# tfhd = tfhd.rename(columns={'fecha': 'join_date'})
X2a_orig = X2a_orig.rename(columns={0: 'fecha'})

# Realizar el rolling join con merge_asof
X2a_orig = pd.merge_asof(tfhd, X2a_orig, on='fecha', direction='backward')


# Si quieres que el resultado esté en el DataFrame original (tfhd), puedes hacer lo siguiente:
X2a_orig = pd.merge(tfhd, X2a_orig, on='fecha', how='outer')


In [43]:
import pandas as pd

from datetime import timedelta, datetime

X1_orig = X1_orig.rename(columns={0: "fecha"})
# Filtrar tasas de fondeo

tf_act = X2a_orig.loc[X2a_orig['fecha'] == fval, 1].values[0] / 100
tf_int = X2a_orig.loc[(X2a_orig['fecha'] <= fval) & (X2a_orig['fecha'] >= (fval -  timedelta(int(plazocupon[0])))), 1] / 100
tf_int=tf_int[::-1]  # se hace este cambio para ordenar de mayor a menor


# Ordenar el DataFrame por la columna de unión
X1_orig = X1_orig.sort_values('fecha', ascending=True)

# Agregar la columna join_date al DataFrame X1_orig
X1_orig['join_date'] = X1_orig['fecha']

# Ordenar el DataFrame X2a_orig por la columna de unión
X2a_orig = X2a_orig.sort_values('fecha', ascending=True)

# Hacer el merge (join) de los DataFrames
X2_pr = pd.merge_asof(X1_orig, X2a_orig, on='fecha')
# X2a_orig = pd.merge(X2a_orig, X2_pr, on='fecha', how='outer')

In [50]:
import numpy as np

# Inicialización de variables
m = len(plazos)
N = (plazos // plazocupon) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos = np.zeros(np.sum(N))
contratosT = np.zeros(np.sum(N))
nominalT = np.zeros(np.sum(N))
plazocuponT = np.zeros(np.sum(N))
tasafijaT = np.zeros(np.sum(N))
ulNomT = np.zeros(np.sum(N))

# Cálculos adicionales
plazini = plazos - plazocupon * (N - 1)
ddv = plazocupon - plazini
tfcupon = np.zeros(m)
tfcupondev = np.zeros(m)
tfcupgen = ((1 + tf_act / 360)**(plazocupon[0]) - 1) * 360 / plazocupon[0]

# Bucle para calcular tasas de cupón
for j in range(m):  # en Python los índices comienzan en 0
    tfcupondev[j] = ((np.prod(1 + tf_int[0:ddv[j]] / 360) - 1) * 360) / ddv[j]
    tfcupon[j] = (((1 + tfcupondev[j] * ddv[j] / 360) * (1 + tf_act / 360)**(plazocupon[0] - ddv[j]) - 1) * 360) / plazocupon[0]

print(tfcupondev)
print(tfcupon)

[0.07804226 0.0769081  0.07764526 0.07830053 0.07661949]
[0.07941412 0.07827887 0.07904407 0.07956142 0.07755682]


In [89]:
import numpy as np
from scipy.interpolate import interp1d

# Suponemos que m, N, plazos, plazocupon, contratos, tfcupon, tfcupgen, etc. ya están definidos
# También asumimos que nodos, X_orig, nodos3, X3_orig y itpl están definidos

# Inicialización
VTplazos = np.zeros(np.sum(N))
contratosT = np.zeros(np.sum(N))
plazocuponT = np.zeros(np.sum(N))
tasafijaT = np.zeros(np.sum(N))
ulNomT = np.zeros(np.sum(N))
Xvp = np.zeros((n, len(VTplazos)))
Xst = np.zeros((n, len(VTplazos)))

for j in range(m):
    sum_N = np.sum(N[:j+1])
    sum_N_prev = np.sum(N[:j]) if j > 0 else 0

    if j == 0:
        VTplazos[:sum_N] = np.arange(plazini[j], plazos[j] + 1, plazocupon[j])
        contratosT[:sum_N] = contratos[j]
        plazocuponT[:sum_N] = plazocupon[j]
        ulNomT[sum_N - 1] = contratos[j]
        tasafijaT[0] = tfcupon[j]
        tasafijaT[1:sum_N] = tfcupgen
    else:
        VTplazos[sum_N_prev:sum_N] = np.arange(plazini[j], plazos[j] + 1, plazocupon[j])
        contratosT[sum_N_prev:sum_N] = contratos[j]
        plazocuponT[sum_N_prev:sum_N] = plazocupon[j]
        tasafijaT[sum_N_prev] = tfcupon[j]
        tasafijaT[sum_N_prev + 1:sum_N] = tfcupgen
        ulNomT[sum_N - 1] = contratos[j]

for i in range(n):
    if itpl == 0:
        interpolator = interp1d(nodos, X_orig.iloc[i, :-1], kind='linear', fill_value="interpolate")
        Xvp[i, :] = interpolator(VTplazos)

        interpolator3 = interp1d(nodos3, X3_orig.iloc[i, :], kind='linear', fill_value="interpolate")
        Xst[i, :] = interpolator3(VTplazos)
    else:
        # Código para interpolación 'talamb' (no definido en el ejemplo de R original)
        pass


In [90]:
import numpy as np

def bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N):
    V0 = np.zeros(len(N))  # Inicializa V0, equivalente a matrix(0, 1, nrow(as.matrix(N))) en R
    V0f = (((contratosT * tasafijaT * (plazocuponT / 360)) + ulNomT) / (1 + (Xvp + Xst) * VTplazos / 360)) * nominal

    for j in range(len(N)):  # Cambia índices de 1:nrow(as.matrix(N)) a 0:(len(N)-1)
        if j == 0:  # Cambia de j==1 a j==0
            V0[j] = np.sum(V0f[:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[:j]):np.sum(N[:j+1])])

    return V0

# Suponiendo que contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, y N ya están definidos
V0 = bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[0, :], Xst[0, :], N)  # Cambia de Xvp[1, ] y Xst[1, ] a Xvp[0, :] y Xst[0, :]

print(V0)


[ 2155549.18937959 -2834587.18559718  2815349.56363984 -4344977.24106617
   930116.72280706]
